# Hatespeech classification pipeline

In [58]:
import torch
from hatedetection import BertForSequenceMultiClassification
from hatedetection.preprocessing import preprocess_tweet
from hatedetection.categories import extended_hate_categories
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.nn import functional as F

class HateClassificationOutput:
    """
    Dummy class for classification output
    """
    def __init__(self, hateful, calls_to_action=None, categories=[]):
        """
        Constructor
        """
        self.hateful = hateful
        self.calls_to_action = calls_to_action
        self.categories = categories
    
    def __repr__(self):
        ret = f"{self.__class__.__name__}"
        
        if not self.hateful:
            ret += "(hateful=False)"
            return ret
        else:
            ret += f"(hateful=True, calls_to_action={self.calls_to_action}, categories={self.categories}"
            return ret
    
class HateSpeechAnalyzer:
    """
    Wrapper to use HS models as black-box
    """
    def __init__(self, base_model_name, category_model_name, use_context=False):
        """
        Constructor for HateSpeechAnalyzer class
        """
        self.tokenizer = AutoTokenizer.from_pretrained(base_model_name)
        self.base_model = AutoModelForSequenceClassification.from_pretrained(
            base_model_name, num_labels=2,
        )
        

        self.category_model = BertForSequenceMultiClassification.from_pretrained(
            category_model_name, num_labels=len(extended_hate_categories)
        )

        max_length = 256 if use_context else 128
        
        self.tokenizer.model_max_length = max_length
        

    def predict(self, sentence, context=None):
        """
        
        """
        device = self.base_model.device

        args = []


        # If context, prepend it
        if context:
            args.append(context)
        args.append(preprocess_tweet(sentence))

        idx = self.tokenizer.encode(*args)
        # Reshape to be (1, L) and send to model d
        idx = torch.LongTensor(idx).view(1, -1).to(device)
        
        outs = self.base_model(idx)
        hateful = bool(outs.logits.argmax().item())
        
        if hateful:
            """
            Look for categories
            """
            category_output = self.category_model(idx)
            category_output = category_output.logits.detach().cpu().numpy()[0]

            # If logit of cat is > 0 this means sigmoid(logit) > 0.5.
            categories = [cat for cat, out in list(zip(extended_hate_categories, category_output > 0)) if out]
            
            calls_to_action = "CALLS" in categories
            if calls_to_action:
                categories.remove("CALLS")
            
            return HateClassificationOutput(hateful=hateful, calls_to_action=calls_to_action, categories=categories)
        
        return HateClassificationOutput(hateful=hateful)
    

In [59]:
analyzer = HateSpeechAnalyzer("../models/bert-non-contextualized-hate-speech-es/", "../models/bert-non-contextualized-hate-category-es/")

In [61]:
analyzer.predict("Amarillos roñosos de mierda, mátenlos")

HateClassificationOutput(hateful=True, calls_to_action=True, categories=['RACISM']

In [62]:
analyzer.predict("Judíos roñosos de mierda, mátenlos")

HateClassificationOutput(hateful=True, calls_to_action=True, categories=['RACISM']

In [63]:
analyzer.predict("Rusos de mierda, mátenlos")

HateClassificationOutput(hateful=True, calls_to_action=True, categories=['RACISM']

In [67]:
analyzer.predict("Hay que matar a todos las kukas")

HateClassificationOutput(hateful=True, calls_to_action=False, categories=['POLITICS']

In [68]:
analyzer.predict("Hay que matar a las cucarachas")

HateClassificationOutput(hateful=True, calls_to_action=True, categories=['POLITICS']

In [71]:
analyzer.predict("Maten a las feministas")

HateClassificationOutput(hateful=True, calls_to_action=True, categories=['WOMEN', 'CRIMINAL']

In [75]:
analyzer.predict("Un par de balazos a esos negros")

HateClassificationOutput(hateful=True, calls_to_action=True, categories=['RACISM', 'CRIMINAL']

In [76]:
analyzer.predict("Una pala para esos vagos")

HateClassificationOutput(hateful=True, calls_to_action=False, categories=['CLASS']

# Contextualizado

In [65]:
contextualized_analyzer = HateSpeechAnalyzer(
    "../models/bert-contextualized-hate-speech-es/", "../models/bert-contextualized-hate-category-es/", use_context=True
)

In [84]:
def show_context_and_comments(analyzer, context, comments):
    print(f"Contexto: {context}", "\n"*2)
    print("Comentarios:", "\n")
    
    for i, comment in enumerate(comments):
        print(f"{i:<3} -- {comment}")
        prediction = analyzer.predict(comment, context=context)
        if prediction.hateful:
            rep = "Discriminatorio"
            
            if prediction.calls_to_action:
                rep+= " y llama a la acción"
                
            rep += " contra " + ' - '.join(prediction.categories)
            
            print(rep)
        else:
            print("No discriminatorio")

context = "Motín de presos en la cárcel de Devoto"

comments = [
    "Negro chorro de mierda",
    "Este país está lleno de vagos",
    "Cuarenta ladrillos y esto se arregla...",
    "Bala",
    "Videla volvé!",
    "Ahhhn y donde estan las feministas de mierda eh?",
    "Y qué van a decir los chinos sobre este desastre?",
]

show_context_and_comments(contextualized_analyzer, context, comments)

Contexto: Motín de presos en la cárcel de Devoto 


Comentarios: 

0   -- Negro chorro de mierda
HateClassificationOutput(hateful=True, calls_to_action=False, categories=['RACISM', 'CRIMINAL']
Discriminatorio contra RACISM - CRIMINAL
1   -- Este país está lleno de vagos
HateClassificationOutput(hateful=False)
No discriminatorio
2   -- Cuarenta ladrillos y esto se arregla...
HateClassificationOutput(hateful=False)
No discriminatorio
3   -- Bala
HateClassificationOutput(hateful=True, calls_to_action=True, categories=['CRIMINAL']
Discriminatorio y llama a la acción contra CRIMINAL
4   -- Videla volvé!
HateClassificationOutput(hateful=False)
No discriminatorio
5   -- Ahhhn y donde estan las feministas de mierda eh?
HateClassificationOutput(hateful=False)
No discriminatorio
6   -- Y qué van a decir los chinos sobre este desastre?
HateClassificationOutput(hateful=False)
No discriminatorio
